In [1]:
import numpy as np
import os
os.chdir('/Users/magictavern/Documents/kaggkle/all')
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
test = pd.read_csv('test.csv')
train= pd.read_csv('train.csv')
full=pd.concat([train,test],ignore_index=True)

/Users/magictavern/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
full.Age=full.Age.fillna(full.Age.mean())
full.Fare=full.Fare.fillna(full.Fare.mean())
print(full.Embarked.value_counts())

S    916
C    270
Q    123
Name: Embarked, dtype: int64


In [6]:
full.Embarked=full.Embarked.fillna('S')
full.Cabin=full.Cabin.fillna('S')
full.Cabin=full.Cabin.map(lambda x: x[0])
print(full.Cabin.value_counts())

S    1014
C      94
B      65
D      46
E      41
A      22
F      21
G       5
T       1
Name: Cabin, dtype: int64


In [7]:
CabinDf=pd.get_dummies(full.Cabin,prefix='Cabin')
EmbarkedDf=pd.get_dummies(full.Embarked,prefix='Embarked')
def GetT(x):
    y=x.split(',')[1]
    z=y.split('.')[0]
    return z.strip( )#去除空格
full.Name=full.Name.map(GetT)
title_mapDict={ 'Capt':'Officer', 'Col':'Officer','Major':'Officer','Dr':'Officer','Don':'Royalty','Jonkeer':'Royalty','Sir':'Royalty','Rev':'Officer',
               'the Countess':'Royalty','Mme':'Mrs','Mile':'Miss','Ms':'Mrs','Mr':'Mr','Mrs':'Mrs','Miss':'Miss','Master':'Master','Lady':'Royalty','Dona':'Royalty'}
full.Name=full.Name.map(title_mapDict)
NameDf=pd.get_dummies(full.Name,prefix='Name')
PclassDf=pd.get_dummies(full.Pclass,prefix='Pclass')
SexDf=pd.get_dummies(full.Sex)

In [8]:
FamilyDf=pd.DataFrame()
FamilyDf['FamilySize']=full.SibSp+full.Parch+1
print(FamilyDf.FamilySize.value_counts())
FamilyDf['FamilySingle']=FamilyDf.FamilySize.map( lambda x : 1 if x<=1 else 0)
FamilyDf['FamilySmall']=FamilyDf.FamilySize.map( lambda x : 1 if 1 <x<=3 else 0)
FamilyDf['FamilyLarge']=FamilyDf.FamilySize.map( lambda x : 1 if  x>3 else 0)
full=pd.concat([full,CabinDf,EmbarkedDf,NameDf,PclassDf,FamilyDf,SexDf],axis=1)
full.drop(['Cabin','Embarked','Name','Pclass','Sex'],axis=1,inplace=True)
print(full.corr().Survived.abs().sort_values())
full.drop(columns='Ticket',inplace=True)

1     790
2     235
3     159
4      43
6      25
5      22
7      16
11     11
8       8
Name: FamilySize, dtype: int64
Embarked_Q      0.003650
PassengerId     0.005007
Cabin_G         0.016040
FamilySize      0.016639
Cabin_A         0.022287
Cabin_T         0.026456
FamilyLarge     0.029945
Name_Officer    0.031316
SibSp           0.035322
Name_Royalty    0.050561
Cabin_F         0.057935
Age             0.070323
Parch           0.081629
Name_Master     0.085221
Pclass_2        0.093349
Cabin_C         0.114652
Cabin_E         0.145321
Embarked_S      0.149683
Cabin_D         0.150716
Embarked_C      0.168240
Cabin_B         0.175095
FamilySingle    0.203367
FamilySmall     0.238059
Fare            0.257307
Pclass_1        0.285904
Cabin_S         0.316912
Pclass_3        0.322308
Name_Miss       0.327093
Name_Mrs        0.344935
female          0.543351
male            0.543351
Name_Mr         0.549199
Survived        1.000000
Name: Survived, dtype: float64


In [9]:
X=full.loc[:890,full.columns!='Survived']
y=full.loc[:890,'Survived']

In [11]:
param_test1 = {'n_neighbors':range(2,10),'weights':['uniform', 'distance']}
gsearch1 = GridSearchCV(estimator = KNeighborsClassifier(), 
                       param_grid = param_test1, scoring='roc_auc',iid=True,cv=10)
gsearch1.fit(X,y)
print('网格搜索-最佳度量值:',gsearch1.best_score_)  
print('网格搜索-最佳参数：',gsearch1.best_params_)  
print('网格搜索-最佳模型：',gsearch1.best_estimator_) 

网格搜索-最佳度量值: 0.5420950109799387
网格搜索-最佳参数： {'n_neighbors': 8, 'weights': 'distance'}
网格搜索-最佳模型： KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=8, p=2,
           weights='distance')


In [12]:
param_test1 = {'n_estimators':range(100,1000,100)}
gsearch1 = GridSearchCV(estimator = RandomForestClassifier(), 
                       param_grid = param_test1, scoring='roc_auc',cv=10)
gsearch1.fit(X,y)
print('网格搜索-最佳度量值:',gsearch1.best_score_)  
print('网格搜索-最佳参数：',gsearch1.best_params_)  
print('网格搜索-最佳模型：',gsearch1.best_estimator_) 

网格搜索-最佳度量值: 0.8706058326016073
网格搜索-最佳参数： {'n_estimators': 500}
网格搜索-最佳模型： RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


In [14]:
param_test2 = {'max_depth':range(3,14,2),'max_features':np.arange(0.1,1,0.2)}
gsearch2 = GridSearchCV(estimator = RandomForestClassifier(n_estimators= 500),
   param_grid = param_test2, scoring='roc_auc', cv=10)
gsearch2.fit(X,y)
print('网格搜索-最佳度量值:',gsearch2.best_score_) 
print('网格搜索-最佳参数：',gsearch2.best_params_)  
print('网格搜索-最佳模型：',gsearch2.best_estimator_)  

网格搜索-最佳度量值: 0.87254137447245
网格搜索-最佳参数： {'max_depth': 7, 'max_features': 0.30000000000000004}
网格搜索-最佳模型： RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=7, max_features=0.30000000000000004,
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False)


In [16]:
RFC_Model=gsearch2.best_estimator_
RFC_Model.fit(X,y)
pred_X=full.loc[891:,full.columns!='Survived']
pred_y=pd.DataFrame(RFC_Model.predict(pred_X),index=pred_X.PassengerId)
pred_y.rename(columns={0:'Survived'},inplace=True)
pred_y.reset_index(inplace=True)
pred_y=pred_y.astype(int)
pred_y.to_csv('titanic_pred6.csv',index=False)